### Unsupervised GRU-VAE training  

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np # For standard deviation calculation
from modbus import ModbusDataset,CSVDataset
from sklearn.metrics import f1_score, accuracy_score
import torch.optim as optim

def false_discovery_rate(y_true, y_pred):
    """
    Calculates the False Discovery Rate.
    """
    true_positives = torch.sum((y_pred == 1) & (y_true == 1)).float()
    false_positives = torch.sum((y_pred == 1) & (y_true == 0)).float()
    if true_positives + false_positives == 0:
        return 0.0
    return (false_positives / (true_positives + false_positives)).item()


def train_ae(model, dataloader, num_epochs=10):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    model.train()
    for epoch in range(num_epochs):
        for data in dataloader:
            inputs, _ = data
            inputs = inputs.float()
            # ===================forward=====================
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

def train_vae(model, dataloader, num_epochs=10):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    model.train()
    for epoch in range(num_epochs):
        for data in dataloader:
            inputs, _ = data
            inputs = inputs.float()
            # ===================forward=====================
            recon_batch, mu, logvar = model(inputs)
            recon_loss = nn.functional.mse_loss(recon_batch, inputs, reduction='sum')
            kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            loss = recon_loss + kld_loss
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



def compute_threshold(mse_values):
    """
    Computes the anomaly detection threshold (for marking sample as Intrusion if the IS was greater )
    based on the mean and standard deviation of Mean Squared Error (MSE) values.
    Formula: thr = mean(MSE) + std(MSE)

    Args:
        mse_values (torch.Tensor or list/np.array): A tensor or list of MSE values
                                                    obtained from the validation set.

    Returns:
        float: The calculated threshold.
    """
    if not isinstance(mse_values, torch.Tensor):
        mse_values = torch.tensor(mse_values, dtype=torch.float32)

    if mse_values.numel() == 0:
        return 0.0 
    mean_mse = torch.mean(mse_values)
    std_mse = torch.std(mse_values)

    threshold = mean_mse + std_mse
    return threshold.item() 





--- Testing Threshold Computation ---
0.16264086961746216


In [2]:

# AutoEncoder (AE)
class AE(nn.Module):
    """
    Encoder: (87-64-32)
    Decoder: (32-64-87)
    """
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(87, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 87),
            nn.ReLU()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

# Variational AutoEncoder (VAE)
class VAE(nn.Module):
    """
    Encoder: (87-64-64-32 for mu and log_var)
    Decoder: (32-64-64-87)
    return x_recon, mu, logvar
    """
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(87, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(64, 32)
        self.fc_logvar = nn.Linear(64, 32)
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 87),
            nn.ReLU()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar


# Adversarial AutoEncoder (AAE)
class AAE(nn.Module):
    """
    Encoder:(87-16-4-2)
    Decoder: (2-4-16-87)
    Discriminator: (16-4-2)
    Activation: LeakyReLU
    """
    def __init__(self):
        super(AAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(87, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 2),
            nn.LeakyReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 87),
            nn.LeakyReLU()
        )
        self.discriminator = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 2), # Output for binary classification (real/fake)
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z

    def discriminate(self, z):
        return self.discriminator(z)
# GRU-VAE
class GRUVAE(nn.Module):
    """
    Gated Recurrent Unit : num_layers=2, hidden_size=256, dropout=0.01,window size (seq_len)= 40
    """
    def __init__(self, input_dim=87, hidden_dim=256, latent_dim=32, num_layers=2, dropout=0.01):
        super(GRUVAE, self).__init__()
        self.encoder_gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        self.fc_z_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.decoder_gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # x shape: [batch_size, seq_len=40, input_dim=87]
        _, hidden = self.encoder_gru(x)  # hidden: [num_layers, batch_size, hidden_dim] h0 Defaults to zeros (not provided).
        h = hidden[-1]  # [batch_size, hidden_dim]
        mu = self.fc_mu(h)  # [batch_size, latent_dim]
        logvar = self.fc_logvar(h)  # [batch_size, latent_dim]
        z = self.reparameterize(mu, logvar)  # [batch_size, latent_dim]
        h0 = self.fc_z_to_hidden(z).unsqueeze(0).repeat(self.encoder_gru.num_layers, 1, 1)  # [num_layers, batch_size, hidden_dim]
        # Initialize decoder input with zeros 
        decoder_input = torch.zeros_like(x)
        output, _ = self.decoder_gru(decoder_input, h0)  # [batch_size, seq_len, hidden_dim]
        x_recon = self.fc_out(output)  # [batch_size, seq_len, input_dim]
        return x_recon, mu, logvar

In [4]:
dataset_directory = "./dataset" # change this to the folder contain benign and attack subdirs
modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and